In [ ]:
import numpy as np
from scipy import ndimage, sparse
from scipy.linalg import eigh, inv, logm, norm
import numpy as np
import matplotlib.pyplot as plt
import sys

def airm(a,b): #affine invariant Riemannian metric
    w, v = eigh(a)
    wsqrt = np.sqrt(np.abs(w))
    a1 = inv(v @ np.diag(wsqrt) @ v.T)
    L = norm( logm( a1 @ b @ a1 ) )
    return L

def lerm(a,b): #log euclidean Riemannian metric
    L = norm( logm(a) - logm(b) )
    return L

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import networkx as nx
from holoviews import opts

hv.extension('bokeh')

defaults = dict(width=400, height=400, padding=0.1)
hv.opts.defaults(
    opts.EdgePaths(**defaults), opts.Graph(**defaults), opts.Nodes(**defaults))

In [ ]:
sub=str(9)
r=str(0)
c=str(8)

In [ ]:
filename = '/home/garner1/Work/dataset/tissue2graph/npz/iMS266_20190426_001.sub'+sub+'.tif_data_r'+r+'_c'+c+'.npz'
data = np.load(filename,allow_pickle=True)

covds = data['descriptors']
sintensity = data['sintensity']
row=data['row'];col=data['col']
XYs=data['centroids']

In [ ]:
print(XYs.shape,covds.shape)

In [ ]:
X = []
for ind in range(len(covds)):
    b = logm(covds[ind]).flatten()
    X.append(b)
X = np.asfarray(X)

In [ ]:
import umap
import warnings
warnings.filterwarnings('ignore')

mat_X = umap.umap_.fuzzy_simplicial_set(
    X,
    n_neighbors=50, 
    random_state=np.random.RandomState(seed=42),
    metric='l2',  
    metric_kwds={},
    knn_indices=None,
    knn_dists=None,
    angular=False,
    set_op_mix_ratio=1.0,
    local_connectivity=2.0,
    verbose=False
    )

In [ ]:
mat_XY = umap.umap_.fuzzy_simplicial_set(
    XYs,
    n_neighbors=50, 
    random_state=np.random.RandomState(seed=42),
    metric='l2',
    metric_kwds={},
    knn_indices=None,
    knn_dists=None,
    angular=False,
    set_op_mix_ratio=1.0,
    local_connectivity=2.0,
    verbose=False
    )

In [ ]:
import networkx as nx
import seaborn as sns; sns.set()

sns.set(style='white', rc={'figure.figsize':(50,50)})
# img = cv2.imread('/home/garner1/Work/dataset/dataset_patient52/iMS266_20190426_001.sub'+str(img_id)+'.jpg', cv2.IMREAD_GRAYSCALE)

G = nx.from_scipy_sparse_matrix(mat_X) # if sparse matrix
eset = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] > 0.0]
weights = [d['weight'] for (u, v, d) in G.edges(data=True)]
pos = XYs

nx.draw_networkx_nodes(G, pos,alpha=1.0)
nx.draw_networkx_edges(G, pos, edgelist=eset,alpha=1.0, width=weights,edge_color='r',style='solid')
plt.axis('off')
# plt.savefig(str(img_id)+'graph_only.png',bbox_inches='tight')
# plt.close()


In [ ]:
sns.set(style='white', rc={'figure.figsize':(50,50)})

G = nx.from_scipy_sparse_matrix(mat_XY) # if sparse matrix
eset = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] > 0]
weights = [d['weight'] for (u, v, d) in G.edges(data=True)]
pos = XYs

nx.draw_networkx_nodes(G, pos,alpha=1.0)
nx.draw_networkx_edges(G, pos, edgelist=eset,alpha=1.0, width=weights,edge_color='r',style='solid')
plt.axis('off')

In [ ]:
def edges_rescaling(edges): # edges are mat.data where mat is a sparse scipy matrix
    edges = np.log10(edges) # log rescale weights because they vary over many decades
    edges -= min(edges)-5 # make them positive 
#     edges = np.log1p(edges)
    edges /= max(edges)*1.0/10 # rescale from 0 to 10
    return edges

In [ ]:
hada = mat_X.multiply(mat_XY)

In [ ]:
sns.set(style='white', rc={'figure.figsize':(10,10)})
sns.distplot(np.log(hada.data))

In [ ]:
hada.data = edges_rescaling(hada.data) # rescale to a log-scale from 0 to 10

In [ ]:
sns.set(style='white', rc={'figure.figsize':(10,10)})
sns.distplot(hada.data)

In [ ]:
sns.set(style='white', rc={'figure.figsize':(50,50)})

G = nx.from_scipy_sparse_matrix(hada)
eset = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] > 0.0]
weights = [d['weight'] for (u, v, d) in G.edges(data=True)]
pos = XYs

nx.draw_networkx_nodes(G, pos,alpha=1.0)
nx.draw_networkx_edges(G, pos, edgelist=eset,alpha=1.0, width=weights,edge_color='r',style='solid')
# nx.draw_networkx_edges(G, pos, edgelist=eset,alpha=1.0, edge_color='r',style='solid')
plt.axis('off')

In [ ]:
import tensorly as tl
from tensorly.decomposition import non_negative_parafac

T = np.zeros(shape = (X.shape[0],X.shape[0],2))
T[:,:,0] = hada.toarray() 
# T[:,:,1] = mat_X.toarray()
T[:,:,1] = mat_XY.toarray()
sys.getsizeof(T)

In [ ]:
factors = non_negative_parafac(T, rank=3, verbose=1, n_iter_max=10,tol=1e-08,init='random')

In [ ]:
# [factors[1][ind].shape for ind in range(3)]

In [ ]:
for i in range(factors[1][0].shape[1]):
    a = factors[1][0][:,i]
    b = factors[1][1][:,i]
    c = factors[1][2][:,i]
    s = 0.5*(a+b)
    mat = np.outer(s,s) # symmetrize wrt a & b
    print(tl.norm(a,2)*tl.norm(b,2)*tl.norm(c,2))
#     plt.imshow(mat, cmap='Blues', interpolation='nearest')
    plt.figure(figsize=(10, 10))
    sns.heatmap(mat,cmap='Blues',square=True,xticklabels=False,yticklabels=False)
    plt.show() 

In [ ]:
factors[1][0][:10,:3]

In [ ]:
factors[1][1][:10,:3]